<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
!wget https://raw.githubusercontent.com/DmitryKutsev/ml_hw/master/small_data.csv


--2020-03-16 13:18:03--  https://raw.githubusercontent.com/DmitryKutsev/ml_hw/master/small_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24394136 (23M) [text/plain]
Saving to: ‘small_data.csv.3’

small_data.csv.3    100%[===================>]  23.26M  69.6MB/s    in 0.3s    

2020-03-16 13:18:04 (69.6 MB/s) - ‘small_data.csv.3’ saved [24394136/24394136]



In [0]:
import pandas as pd
from sklearn import cluster
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AffinityPropagation, MeanShift, AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD, NMF

In [0]:

from sklearn.cluster import AffinityPropagation, AgglomerativeClustering, DBSCAN, \
                            KMeans, MiniBatchKMeans, Birch, MeanShift, SpectralClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, \
                            silhouette_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn.datasets import make_blobs

In [0]:
data = pd.read_csv('small_data.csv')

In [80]:
data.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,793dd3d5f0f0,9467e0341c0e,Алтайский край,Барнаул,Личные вещи,Детская одежда и обувь,Для девочек,Платья и юбки,146-152 см (10-12 лет),Платье праздничное для девочки,Очень красивое торг,1500.0,106,2017-04-12,Company,ace7b2b7f066a6b6ff244f90d030c2c1a1bf118dd0e1ac...,573.0
1,f8c965d9b668,1905516660f5,Свердловская область,Каменск-Уральский,Для дома и дачи,Мебель и интерьер,"Кровати, диваны и кресла",NaN,NaN,Кровать двухспальная,Двуспальная кровать 140х200 с металлическим ка...,5700.0,10,2017-04-15,Private,b38ef24dd5b876ed626bd98241e228e4e19e4160d33abe...,1439.0
2,2f153c00d561,b4dc4849fdd4,Самарская область,Самара,Бытовая электроника,Телефоны,Samsung,NaN,NaN,Samsung S3 mini gt-i8190,"Продам телефон самсунг 2 ядра, 1 гиг оперативн...",1500.0,6,2017-04-12,Private,8530cc0c8d5dc6d212497dffd610d89b92a7db952934db...,2932.0
3,7fc5eea56915,02c14d39fdb4,Саратовская область,Саратов,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,44–46 (M),"Красивое платье на одно плечо, в идеале размер м","Заказывала в Сша, прекрасное качество, в деле ...",1000.0,1280,2017-04-13,Company,eb21745ba276d3b00b05856a58fe719e31a82419c0221a...,386.0
4,b01000d29ea3,b17f14110367,Удмуртия,Ижевск,Личные вещи,Детская одежда и обувь,Для мальчиков,"Шапки, варежки, шарфы",NaN,Стильная панама,Состояние новой. Размер до года- полтора. По ...,150.0,145,2017-04-14,Company,5bf1001fa681b96f121ad8047c85e3747f3dac37830dd0...,209.0


In [0]:
data = data[['category_name', 'title']]

In [82]:
data.shape

(37339, 2)

In [0]:
sample = data.sample(frac=0.1)

In [84]:
sample.shape

(3734, 2)

In [0]:
#################################################

In [0]:

cv = CountVectorizer(max_features=500)
X = cv.fit_transform(sample['title'])

y = sample['category_name']

In [87]:
cluster = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster.fit(X)

Converged after 227 iterations.


AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
                    damping=0.7, max_iter=400, preference=-5, verbose=2)

In [0]:
labels = cluster.labels_

In [89]:
n_clusters = len(set(labels))
n_clusters

207

In [90]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Silhouette Coefficient: 0.340


In [91]:
# эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(y, labels))

Homogeneity: 0.505
Completeness: 0.389
V-measure: 0.439
Adjusted Rand Index: 0.001
Adjusted Mutual Information: 0.319


In [0]:
sample['cluster'] = cluster.labels_

In [105]:
sample[sample.cluster==19].head(10)

,category_name,title,cluster
31728,Оргтехника и расходники,Оболочка магнитного вала Delacamp HP 1005/1102,19
2863,Оргтехника и расходники,HP DeskJet 2130,19
2251,Ноутбуки,HP,19
19724,Оргтехника и расходники,Неисправный лазерный принтер HP LJ 1200,19
30656,Ноутбуки,"Игровой HP c i7, гарантия",19
24574,Товары для компьютера,Клавиатура для ноутбука HP Model LX8,19


In [94]:
sample[sample.category_name=='Телефоны'].cluster.value_counts()

0      45
38     18
113    14
202    10
197     9
151     8
47      8
92      8
7       7
58      7
189     6
72      6
205     5
117     5
84      5
60      4
80      3
107     3
81      1
94      1
52      1
115     1
35      1
146     1
Name: cluster, dtype: int64

In [0]:
########################

In [0]:
def cluster_metrics(cluster, sample, cluster_num, X, y):
  labels = cluster.labels_

  print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X[:10000], labels[:10000]))
  
  # эти три метрки что-то вроде точности, полноты и ф-меры для кластеризации
  print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
  print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
  print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

  print("Adjusted Rand Index: %0.3f"
        % metrics.adjusted_rand_score(y, labels))
  print("Adjusted Mutual Information: %0.3f"
        % metrics.adjusted_mutual_info_score(y, labels))
  
  sample['cluster'] = cluster.labels_
  print(sample[sample.cluster==cluster_num].head(7)) 


In [0]:
sample1 = data.sample(frac=0.01)


In [0]:
cv = CountVectorizer(max_features=500)
X_cv = cv.fit_transform(sample1['title'])

y_cv = sample1['category_name']

In [0]:
tfidf = TfidfVectorizer(max_features=500)
X_tf = tfidf.fit_transform(sample1['title'])
y_tf = sample1['category_name']


In [100]:
cluster1_cv = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster1_cv.fit(X_cv)
labels1_cv = cluster1_cv.labels_

Converged after 104 iterations.


In [101]:
cluster1_tf = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)
cluster1_tf.fit(X_tf)
labels1_tf = cluster1_tf.labels_

Converged after 17 iterations.


In [102]:
cluster_metrics(cluster1_cv, sample1, 11, X_cv, y_cv)

Silhouette Coefficient: -0.023
Homogeneity: 0.462
Completeness: 0.399
V-measure: 0.428
Adjusted Rand Index: 0.066
Adjusted Mutual Information: 0.129
      category_name                             title  cluster
2757       Квартиры  3-к квартира, 88.7 м², 10/16 эт.       11
35998      Квартиры     1-к квартира, 36 м², 3/10 эт.       11
33552      Квартиры     2-к квартира, 54 м², 2/10 эт.       11
32390      Квартиры     1-к квартира, 42 м², 9/10 эт.       11
28453      Квартиры     1-к квартира, 38 м², 5/10 эт.       11
16688      Квартиры     1-к квартира, 37 м², 5/10 эт.       11


In [103]:
cluster_metrics(cluster1_tf, sample1, 11, X_tf, y_tf)

Silhouette Coefficient: -0.026
Homogeneity: 0.331
Completeness: 0.332
V-measure: 0.331
Adjusted Rand Index: 0.065
Adjusted Mutual Information: 0.088
                    category_name                           title  cluster
35020                    Телефоны  Чехол для телефона Alcatel c 7       11
26312                 Фототехника                       Смена 8 М       11
8820   Товары для детей и игрушки               Мобиль в кроватку       11
11631      Детская одежда и обувь                        Колготки       11
10874      Детская одежда и обувь                         Жилетка       11
880                      Телефоны            LG K8 LTE (Гарантия)       11
8329         Билеты и путешествия              Италия в Июне 2017       11


In [0]:
cv = CountVectorizer(max_features=500)
X_cv = cv.fit_transform(sample1['title'])

y_cv = sample1['category_name']

In [0]:
sample = data.sample(frac=0.01)
cv = CountVectorizer(max_features=500)
tfidf = TfidfVectorizer(max_features=500)
svd = TruncatedSVD(50)
nmf = NMF(50)
X_cv = cv.fit_transform(sample['title'])
X_cv = tfidf.fit_transform(sample['title'])
y_cv = sample1['category_name']

X_svd_cv = svd.fit_transform(X_cv)
X_svd_tf = svd.fit_transform(X_tf)

X_nmf_cv = svd.fit_transform(X_cv)
X_nmf_tf = svd.fit_transform(X_tf)


y = sample['category_name']

In [71]:
# cluster2 = AgglomerativeClustering(n_clusters=170)
# cluster2_svd_cv.fit(X_svd_cv)

cluster2_svd_cv = AgglomerativeClustering(n_clusters=170)
cluster2_svd_cv.fit(X_svd_cv)

cluster2_svd_tf = AgglomerativeClustering(n_clusters=170)
cluster2_svd_tf.fit(X_svd_tf)

cluster2_nmf_cv = AgglomerativeClustering(n_clusters=170)
cluster2_nmf_cv.fit(X_nmf_cv)

cluster2_nmf_tf = AgglomerativeClustering(n_clusters=170)
cluster2_nmf_tf.fit(X_nmf_tf)


AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='ward', memory=None, n_clusters=170)

In [72]:
cluster_metrics(cluster2_svd_cv, sample1, 11, X_svd_cv, y_cv)

Silhouette Coefficient: 0.375
Homogeneity: 0.642
Completeness: 0.421
V-measure: 0.508
Adjusted Rand Index: 0.005
Adjusted Mutual Information: 0.012
            item_id       user_id  ... image_top_1 cluster
4156   e5d721e713d8  33e16cf248c0  ...       386.0      11
21064  a3e29296dda1  0c3b1f60f933  ...        86.0      11

[2 rows x 18 columns]


In [73]:
cluster_metrics(cluster2_svd_tf, sample1, 11, X_svd_tf, y_cv)

Silhouette Coefficient: 0.383
Homogeneity: 0.681
Completeness: 0.458
V-measure: 0.548
Adjusted Rand Index: 0.008
Adjusted Mutual Information: 0.097
            item_id       user_id  ... image_top_1 cluster
13407  b5bfff80e18a  77254bec9263  ...      2857.0      11
2848   cae2e42bb22a  607b268b57bb  ...         NaN      11
32303  852eb9650fad  cbb084e04b30  ...      2828.0      11

[3 rows x 18 columns]


In [74]:
cluster_metrics(cluster2_nmf_cv, sample1, 11, X_nmf_cv, y_cv)

Silhouette Coefficient: 0.352
Homogeneity: 0.651
Completeness: 0.422
V-measure: 0.512
Adjusted Rand Index: 0.004
Adjusted Mutual Information: 0.018
            item_id       user_id  ... image_top_1 cluster
23745  469e00166404  6308f528ff7f  ...         NaN      11
5822   2af4ecb68fc5  b37b4fdd738a  ...      2742.0      11

[2 rows x 18 columns]


In [75]:
cluster_metrics(cluster2_nmf_tf, sample1, 1, X_nmf_tf, y_cv)

Silhouette Coefficient: 0.400
Homogeneity: 0.671
Completeness: 0.455
V-measure: 0.542
Adjusted Rand Index: -0.000
Adjusted Mutual Information: 0.084
            item_id       user_id  ... image_top_1 cluster
32368  cd7a51709ffc  4ba882ce8421  ...        88.0       1
21726  d5d3a304b543  624f1cf11df0  ...       639.0       1

[2 rows x 18 columns]


In [0]:
Самый неплохой результат показала пара 